In [1]:
!pip install selenium
!pip install webdriver_manager

In [2]:
from  selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from os.path  import basename
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import re

In [3]:
main_link = 'https://www.tripadvisor.es'

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/carlosperezricardo/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [5]:
driver.get("https://www.tripadvisor.es/Attraction_Review-g187147-d13986330-Reviews-Atelier_des_Lumieres-Paris_Ile_de_France.html#REVIEWS")
time.sleep(4)

In [15]:
# Cookies
try: 
    driver.find_element_by_id("_evidon-accept-button").click()
except:
    pass

In [16]:
# Language
driver.find_element_by_class_name("_2jN6riV4").click()

In [22]:
# Languages
content = driver.page_source
soup = BeautifulSoup(content)

content = driver.page_source
soup = BeautifulSoup(content)

In [23]:
lang_links = soup.findAll(class_="_2U59ljCM")

In [24]:
languages = {}
for link in lang_links:
    #print(link.text)
    #print(link)
    #print(link.attrs['id'])
    languages[link.text] = link.attrs['id']
languages

{'Chino (simplificado)': 'menu-item-zhCN',
 'Noruego': 'menu-item-no',
 'Alemán': 'menu-item-de',
 'Sueco': 'menu-item-sv',
 'Ruso': 'menu-item-ru',
 'Portugués (Portugal)': 'menu-item-pt',
 'Coreano': 'menu-item-ko',
 'Griego': 'menu-item-el',
 'Chino (Taiwán)': 'menu-item-zhTW',
 'Inglés': 'menu-item-en',
 'Italiano': 'menu-item-it',
 'Francés': 'menu-item-fr',
 'Húngaro': 'menu-item-hu',
 'Hebreo': 'menu-item-iw',
 'Español (España)': 'menu-item-es',
 'Checo': 'menu-item-cs',
 'Japonés': 'menu-item-ja',
 'Eslovaco': 'menu-item-sk',
 'Polaco': 'menu-item-pl',
 'Danés': 'menu-item-da',
 'Holandés': 'menu-item-nl'}

In [29]:
# Select a language
#driver.find_element_by_id( languages['Español (España)'] ).click()
driver.find_element_by_id( languages['Inglés'] ).click()
time.sleep(2)

In [31]:
# We get whole reviews.
content = driver.page_source
soup = BeautifulSoup(content)

# Repeat it!
content = driver.page_source
soup = BeautifulSoup(content)

In [32]:
names = soup.findAll(class_="_2nBYkPk3")
names[0]

<div class="_2nBYkPk3"><div class="_1o0NHSNc"><div class="_3t0zrF_f"><div class="HiSmthtM"><div class="_1dkAVM7-" style="z-index: 0;"><div class="f3gJ-yIA _3sXDPf_W"><a aria-hidden="true" class="_7c6GgQ6n _22upaSQN _37QDe3gr" href="/Profile/CA_travelz" tabindex="-1" target="_self"><div class="WullykOU _2L7OTqqK"><picture class="_2f-Th360 _3YW9cIgT" style="width: 32px; height: 32px;"><img alt="Alicia H." height="100" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/f6/e4/ca/default-avatar-2020-51.jpg?w=100&amp;h=-1&amp;s=1" width="100"/></picture></div></a></div></div></div></div><div class="_1svEu5jc"><span class="DrjyGw-P _1SRa-qNz _2AAjjcx8"><a class="_7c6GgQ6n _22upaSQN _37QDe3gr WullykOU _3WoyIIcL" href="/Profile/CA_travelz" target="_self">Alicia H.</a></span><div class="_22DaYeMb"><div class="DrjyGw-P _26S7gyB4 _1dimhEoy"><span class="">6 contribuciones</span></div></div></div></div><div class="_2WOrDRiF"><button aria-label="Haz clic para añadir un vo

In [33]:
users = pd.DataFrame( columns=['user','city','contributions'])

def get_user_info(soup, users):

    names = soup.findAll(class_="_2nBYkPk3")
    for name in names:
        #print(name)
        user_name = name.find(class_='_1SRa-qNz').text
        city_name = name.find(class_='_26S7gyB4').text
        
        #try:
        #    contributions_text = name.find(class_='DrjyGw-P _26S7gyB4 _1dimhEoy').text
        #    print(contributions_text)
        #    contributions = [float(s) for s in re.findall(r'-?\d+\.?\d*', contributions_text)][0]
        #    
        #    city_name = city_name.replace(contributions_text,"")
        #except:
        #    contributions = [float(s) for s in re.findall(r'-?\d+\.?\d*', city_name)][0]
        #    city_name = ''
        
        contributions = 0

        users = users.append( {'user':user_name, 'city':city_name, 'contributions':contributions}, ignore_index=True)
        #print(user_name)
        #print(city_name)
        #print(contributions)

    return users

users = get_user_info(soup, users)
users

,user,city,contributions
0,Alicia H.,6 contribuciones,0
1,Timo Marquez,"Versalles, Francia2 contribuciones",0
2,milwmomof4,"Milwaukee, WI20 contribuciones",0
3,Yann2002,"Montreal, Canadá207 contribuciones",0
4,Kaylee,"París, Francia209 contribuciones",0
5,Barbara S,"Cracovia, Polonia728 contribuciones",0
6,Francois B,"Zúrich, Suiza39 contribuciones",0
7,Alejandro Q,"Bodega Bay, CA183 contribuciones",0
8,CarolienBAC,"Templeuve, Bélgica5.984 contribuciones",0
9,nolamomtravels,"Nueva Orleans, LA31 contribuciones",0


In [34]:
# Read reviews

content = driver.page_source
soup = BeautifulSoup(content)

texts = soup.findAll(class_="_2tsgCuqy")

In [35]:
reviews = pd.DataFrame( columns=['title','body'])

In [36]:
i = 1
for text in texts:
    if i%2 != 0:
        title = text.text
    else:
        body = text.text
        reviews = reviews.append({'title':title,'body':body}, ignore_index=True)

    i += 1
    #print(text.text,'\n')
    

In [37]:
reviews['user'] = users['user']

reviews

,title,body,user
0,"Didn’t live up to the hype, not worth the $",Still advertising photos looked better than th...,Alicia H.
1,Nice 1hr show to enjoy with family,A nice show with good music during the present...,Timo Marquez
2,Unique,The hour long program continues on a loop so y...,milwmomof4
3,Different perspective than traditionnal museums,"Great Dali and Gaudi show.It's worth a trip, t...",Yann2002
4,Different way to view art,Nice way to view art in a different way. When ...,Kaylee
5,Dali & Gaudi,We attended two shows - Gaudi & Dali - with on...,Barbara S
6,Atelier des lumières new cluster in Paris,Too many peopleNo enforcement of barrier rules...,Francois B
7,Surrounded by Impresive painters,We have visited this venue several times to en...,Alejandro Q
8,Highly recommended,The visit to Atelier des Lumières truly was on...,CarolienBAC
9,Amazing Experience,We saw this exhibit in November 2019. it was t...,nolamomtravels


In [38]:
def get_reviews(soup, reviews):
    texts = soup.findAll(class_="_2tsgCuqy")
    
    i = 1
    for text in texts:
        if i%2 != 0:
            title = text.text
        else:
            body = text.text
            reviews = reviews.append({'title':title,'body':body}, ignore_index=True)

        i += 1
        #print(text.text,'\n')

    return reviews 

In [39]:
reviews.iloc[0]['body']

'Still advertising photos looked better than the exhibit. Music doesn’t match movement, and not a very impactful show. Would have rather skipped, would not recommend.'

In [40]:
href_class = soup.find(class_="_3djM0GaD")
href = href_class.find('a')['href']
href

'/Attraction_Review-g187147-d13986330-Reviews-or10-Atelier_des_Lumieres-Paris_Ile_de_France.html'

In [41]:
# Go to Next Page
driver.get(main_link+href)
time.sleep(4)

In [42]:
# Do the same 
# We get whole reviews.
content = driver.page_source
soup = BeautifulSoup(content)

users = get_user_info(soup, users)
users

,user,city,contributions
0,Alicia H.,6 contribuciones,0
1,Timo Marquez,"Versalles, Francia2 contribuciones",0
2,milwmomof4,"Milwaukee, WI20 contribuciones",0
3,Yann2002,"Montreal, Canadá207 contribuciones",0
4,Kaylee,"París, Francia209 contribuciones",0
5,Barbara S,"Cracovia, Polonia728 contribuciones",0
6,Francois B,"Zúrich, Suiza39 contribuciones",0
7,Alejandro Q,"Bodega Bay, CA183 contribuciones",0
8,CarolienBAC,"Templeuve, Bélgica5.984 contribuciones",0
9,nolamomtravels,"Nueva Orleans, LA31 contribuciones",0


In [43]:
content = driver.page_source
soup = BeautifulSoup(content)        

In [44]:
href_class = soup.find(class_="_3djM0GaD")
href = href_class.find('a')['href']
href

'/Attraction_Review-g187147-d13986330-Reviews-Atelier_des_Lumieres-Paris_Ile_de_France.html'

In [45]:
driver.get(main_link+href)
time.sleep(4)
        
content = driver.page_source
soup = BeautifulSoup(content)

users = get_user_info(soup, users)

In [46]:
reviews = get_reviews(soup, reviews)

In [47]:
loop = True
i = 2
while loop == True:
    content = driver.page_source
    soup = BeautifulSoup(content)

    try:   
        href_class = soup.find(class_="_3djM0GaD")
        href = href_class.find('a')['href']
        href
        
        # Go to Next Page
        driver.get(main_link+href)
        time.sleep(4)
            
        content = driver.page_source
        soup = BeautifulSoup(content)
        content = driver.page_source
        soup = BeautifulSoup(content)

        users = get_user_info(soup, users)
        reviews = get_reviews(soup, reviews)

        reviews['user'] = users['user']

        i+=1
        print('Page',i)
    except:
        loop = False 
        print('Leaving while')

Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
Page 66
Page 67
Page 68
Page 69
Page 70
Page 71
Page 72
Page 73
Page 74
Page 75
Page 76
Page 77
Page 78
Page 79
Page 80
Page 81
Page 82
Page 83
Page 84
Page 85
Page 86
Page 87
Page 88
Page 89
Page 90
Page 91
Page 92
Page 93
Page 94
Page 95
Page 96
Page 97
Page 98
Page 99
Page 100
Page 101
Page 102
Page 103
Page 104
Page 105
Page 106
Page 107
Page 108
Page 109
Page 110
Page 111
Page 112
Page 113
Page 114
Page 115
Page 116
Page 117
Page 118
Page 119
Page 120
Page 121
Page 122
Page 123
Page 124
Page 1

In [48]:
users

,user,city,contributions
0,Alicia H.,6 contribuciones,0
1,Timo Marquez,"Versalles, Francia2 contribuciones",0
2,milwmomof4,"Milwaukee, WI20 contribuciones",0
3,Yann2002,"Montreal, Canadá207 contribuciones",0
4,Kaylee,"París, Francia209 contribuciones",0
...,...,...,...
3955,Flandes18,"Guadalajara, México77 contribuciones",0
3956,Sieresobesoapechugas,"Madrid, España1.045 contribuciones",0
3957,GonzaloS,Buenos Aires211 contribuciones,0
3958,bako1973,"Buenos Aires, Argentina59 contribuciones",0


In [49]:
reviews

,title,body,user
0,"Didn’t live up to the hype, not worth the $",Still advertising photos looked better than th...,Alicia H.
1,Nice 1hr show to enjoy with family,A nice show with good music during the present...,Timo Marquez
2,Unique,The hour long program continues on a loop so y...,milwmomof4
3,Different perspective than traditionnal museums,"Great Dali and Gaudi show.It's worth a trip, t...",Yann2002
4,Different way to view art,Nice way to view art in a different way. When ...,Kaylee
...,...,...,...
3945,"Atelier des Lumières,,, sublime!!","Arte inmersivo, maravilloso! Estuve en la expo...",Gabriel l
3946,Una maravilla,Preciosa experiencia. No estaba muy convencida...,Tris Mabel
3947,Quedé maravillado!,"Asistí al espectáculo ""La nuit étoilée"" sobre ...",errthu
3948,Muy buena experiencia para los sentidos,"El lugar está muy bien, la muestra que vimos f...",ctaracena


In [50]:
reviews.to_csv('reviews.csv')

# TRANSLATION

In [53]:
web_link = 'https://translate.systran.net/'

In [51]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
There is no [mac64] chromedriver for browser 93.0.4577 in cache
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.15/chromedriver_mac64.zip
Driver has been saved in cache [/Users/carlosperezricardo/.wdm/drivers/chromedriver/mac64/93.0.4577.15]


In [54]:
driver.get(web_link)
time.sleep(4)

In [62]:
text = 'Muy lindo y  diferente admirar el arte de otra manera.Buena experiencia para los niños, el movimiento y las luces lo hacen muy divertido'

In [67]:
#inputEditor
driver.find_element_by_class_name("form-control").send_keys(text)

In [68]:
# Translation

content = driver.page_source
soup = BeautifulSoup(content)    

#outputEditor

In [69]:
translation = soup.find(id="outputEditor")
translation.text

'Hello, today is a very good day. I want to go out in the afternoon for a walk.Very nice and different to admire art in another way.Good experience for children, movement and lights make it very fun'